# Dataframe Update Pipeline

This checks the RCSB PDB for entries since the last query.  New entries are downloaded, parsed, and added to the existing dataframe.

In [30]:
from datetime import date
import requests
import urllib
from bs4 import BeautifulSoup
import json
import os
import csv
import pandas as pd
import glob
import sys
import subprocess


#
# Run the functions in the final cell to update the PDB dataframe
#

today is: 2022-06-21


In [10]:
#Find Last Update Date

def find_last_update():  
    
    # mm/dd/y
    global today_YYYYMMDD
    today_YYYYMMDD = date.today().strftime("%Y-%m-%d")

    list_of_previous_updates = [file for file in sorted(glob.glob('PDB_list_CSVs/*-additions.txt'))]  #for full set, look in subdirs 

    global last_update
    last_update = list_of_previous_updates[-1][14:24] #"2022-04-11"

    print(" - last PDB database update was:", last_update)


In [11]:
#The json query string is the actual query that works, { -> %3A, : -> %7B etc. I couldn't get url encoding to match.

#last_update is date pulled from a file name; inserted into query
#eg. last_update = "2022-04-11"


def pdb_request_list():
    my_json_query_str = f'%7B"query"%3A%7B"type"%3A"group"%2C"logical_operator"%3A"and"%2C"nodes"%3A%5B%7B"type"%3A"terminal"%2C"service"%3A"text"%2C"parameters"%3A%7B"attribute"%3A"exptl.method"%2C"operator"%3A"exact_match"%2C"negation"%3Afalse%2C"value"%3A"X-RAY%20DIFFRACTION"%7D%7D%2C%7B"type"%3A"terminal"%2C"service"%3A"text"%2C"parameters"%3A%7B"attribute"%3A"rcsb_accession_info.initial_release_date"%2C"operator"%3A"greater"%2C"negation"%3Afalse%2C"value"%3A"{last_update}"%7D%7D%5D%2C"label"%3A"text"%7D%2C"return_type"%3A"entry"%2C"request_options"%3A%7B"paginate"%3A%7B"start"%3A0%2C"rows"%3A100000%7D%2C"scoring_strategy"%3A"combined"%2C"sort"%3A%5B%7B"sort_by"%3A"score"%2C"direction"%3A"desc"%7D%5D%7D%7D'

    #Query PDB using REST API
    #Search for X-Ray structures deposited since last date

    global page
    page = requests.get(f"https://search.rcsb.org/rcsbsearch/v2/query?json={my_json_query_str}")
    #print(f"https://search.rcsb.org/rcsbsearch/v2/query?json={my_json_query_str}")
    print(' - PDB qeury for database update is now complete')

last PDB database update was: 2022-06-03
PDB qeury complete


In [54]:
def new_pdbs_to_csv():
    #Parse response from PDB
    soup = BeautifulSoup(page.text, "html")
    jason = json.loads(soup.body.get_text())

    #json.dumps(jason)
    
    #Generate list of PDB names to populate CSV file

    csv_list = [] 
    for mini_dic in jason['result_set']:
        csv_list.append(mini_dic["identifier"])
        
    #Write CSV File containing IDs of new PBB additions
    with open(f'./PDB_list_CSVs/{today_YYYYMMDD}-new-PDB-additions.txt', 'w') as f:
        write = csv.writer(f)  
        write.writerow(csv_list)

    #list of new PDBs 
    #os.environ['PDB_LIST'] = f'./PDB_list_CSVs/{today_YYYYMMDD}-new-PDB-additions.txt'
    global pdb_csv_list
    pdb_csv_list = f'./PDB_list_CSVs/{today_YYYYMMDD}-new-PDB-additions.txt'
    
    print(f" - there are {len(csv_list)} new PDBs since the last update")


 - last PDB database update was: 2022-06-03
 - PDB qeury for database update is now complete
 - there are 286 new PDBs since the last update
 - the new PDB entries have been downloaded and unzippped


In [67]:
def pdb_getter():
    
    #Fetch the PDBs using the shell script provided by the RCSB PBD
    #https://www.rcsb.org/docs/programmatic-access/batch-downloads-with-shell-script
    subprocess.run(['./PDB_batch_download.sh', '-f', f'{pdb_csv_list}', '-o' './PDB_depot', '-p'], stderr=subprocess.PIPE, text=True)

    #Unzip them
    #Run in shell via Popen to expand wild card
    subprocess.Popen('gunzip -d ./PDB_depot/*pdb.gz', shell = True,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.PIPE,
                     text=True)
    
    
    print(" - the new PDB entries have been downloaded and unzippped")

In [77]:
def new_pdb_dataframer():
    #Populate dataframe of new PDB info
    global df_new
    df_new = pd.DataFrame(columns=["Resolution", "Completeness", "I_sigma", "R_value", "R_free", "Detector", "Det_type", "Optics"])


    #Add date - I can drop anything before... 2002? 20 years is a lot and the early entries are garbage/useless

                                        #only the new PDBs (not yet moved to subdirs)
    for filename in glob.glob('PDB_depot/*.pdb'):  #for full set, look in subdirs 
        with open(os.path.join(os.getcwd(), filename), 'r') as f:
            resolution = completeness = i_sigma = r_value = r_free = detector = det_type = optics = "NULL"  #set as null in case line not present

            pdb_id = filename[-8:-4]
            for line in f:

                #Exit condition, 50% drop in time
                #works without strip? remove other line strips for speed 30% drop in time
                if line.startswith("ATOM"):
                    break

                #Resolution
                if line.startswith("REMARK   3   RESOLUTION RANGE HIGH (ANGSTROMS)"):
                    resolution = line.split(":")[1].strip()

                #Completeness
                if line.startswith("REMARK   3   COMPLETENESS FOR RANGE        (%)"):
                    completeness = line.split(":")[1].strip()         

                #I/sigma
                if line.startswith("REMARK 200  <I/SIGMA(I)> FOR THE DATA SET"):
                    i_sigma = line.split(":")[1].strip()      

                #R-Value
                if line.startswith("REMARK   3   R VALUE     (WORKING + TEST SET)"):
                    r_value = line.split(":")[1].strip()


                #R-Free
                if line.startswith("REMARK   3   FREE R VALUE"):
                    if line.split(":")[0].strip().endswith("VALUE"):  #need to match end here
                        r_free = line.split(":")[1].strip()


                #Detector Model (Pilatus 6M, Eiger 16M etc)
                if line.startswith("REMARK 200  DETECTOR MANUFACTURER"):
                    detector = line.split(":")[1].strip()
                    if ";" in detector:
                        detector = line.split(';')[0].strip()  #some lines contain 2 fields, 2nd redundant

                #Detector Type (Pixel, CCD, etc)
                if line.startswith("REMARK 200  DETECTOR TYPE"):
                    det_type = line.split(":")[1].strip()


                #Optics (Mirrors)
                if line.startswith("REMARK 200  OPTICS"):
                    optics = line.split(":")[1].strip()



            #print(pdb_id, resolution, completeness, i_sigma, r_value, r_free, detector, det_type, optics)
            df_new.loc[pdb_id] = [resolution, completeness, i_sigma, r_value, r_free, detector, det_type, optics]
            
    print(" - new dataframe for concatenation is constructed")


In [88]:
def old_dataframe_updater():

    #Load the existing (full) dataframe of PDB info
    df_full = pd.read_feather('/Users/jesse/TDI_bootcamp/capstone_PDB/dataframe_feather/1_initial_dataframe_163K_full.feather')
    
    #Concat old and new dataframe
    df_catted = pd.concat([df_new.reset_index(), df_full]) 

    #remove old indices column
    df_catted = df_catted.reset_index().drop(['level_0'], axis=1) 
    
    #Write the new, updated PDB DF to feather
    df_catted.to_feather('./dataframe_feather/1_updated_dataframe.feather')
    
    print(""" - the prexisting datframe has been updated and written to this location:\
             \n \t ./dataframe_feather/1_updated_dataframe.feather""")

In [1]:
ls

00_To-Do.ipynb                      3_Data_Regression.ipynb
0_DataFrame_Initial_Assembly.ipynb  Capstone_Flowchart.pdf
1_DataFrame_Update_Pipeline.ipynb   PDB_batch_download.sh*
2_DataFrame_Cleaning.ipynb          README.md


In [86]:
def new_pdb_file_cataloger():
    #Move the new PDB files into the 1xxx/ etc archives
    
    #backticks don't actually need curly braces for $VAR expansion
    #Double quotes do
    
    #command = """for n in $(seq 1 9); do for i in `ls PDB_depot/${n}*.pdb`; do mv $i  "PDB_depot/${n}xxx_pdbs"; done; done"""
    command = """for n in $(seq 1 9); do
    for i in `ls PDB_depot/${n}*.pdb`; do
        mv $i  "PDB_depot/${n}xxx_pdbs";
    done
    done"""
    
    subprocess.Popen(command, shell = True,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.PIPE,
                     text=True)
    
    print(" - the new pdbs have been cataloged")

In [ ]:
#Run these functions to update the dataframe

print("today is:", today_YYYYMMDD)

find_last_update()
pdb_request_list()
new_pdbs_to_csv()
pdb_getter()
new_pdb_dataframer()
old_dataframe_updater()
new_pdb_file_cataloger()